ALTA Shared Task 2018

In [28]:
import numpy as np
import pandas as pd
import nltk

In [52]:
#Preparing dataframe for the training data
train_data=pd.read_csv('train_data.csv')
description=list()
for id in train_data.id:
    with open('patents/'+str(id)+'.txt',encoding='utf8',errors='ignore') as f:
        x=f.read()
        description.append(x.strip())
train_data['description']=description


print(train_data.head())


   id first_ipc_mark_section  \
0   0                      A   
1   1                      G   
2   2                      A   
3   3                      A   
4   4                      D   

                                         description  
0  ABSTRACT\n\n  The disclosure relates to a meth...  
1  ABSTRACT\n\n       A system and method are pro...  
2  ABSTRACT\n\nA media module 10 for use with at ...  
3  ABSTRACT\n\n          A support garment having...  
4  THERMALLY REACTIVE THERMOPLASTIC INTERMEDIATE ...  


In [54]:
#Converting description into tokens
def preprocessor(text):
    __tokenization_pattern = r'''(?x)          # set flag to allow verbose regexps
        \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
      | (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
      | \w+(?:-\w+)*        # words with optional internal hyphens
      | \.\.\.              # ellipsis
      | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
    '''

    ## call it using tokenizer.tokenize
    tokenizer = nltk.tokenize.regexp.RegexpTokenizer(__tokenization_pattern)
    tokens = tokenizer.tokenize(text.lower())
    alphabet_tokens = [token for token in tokens if token.isalpha()]
    en_stopwords = set(nltk.corpus.stopwords.words('english'))
    non_stopwords = [word for word in alphabet_tokens if not word in en_stopwords]
    stemmer = nltk.stem.snowball.SnowballStemmer("english")
    stems = [str(stemmer.stem(word)) for word in non_stopwords]
    
    return stems

train_data['description_tokens'] = train_data['description'].apply(preprocessor)


In [55]:
print(train_data.head())

   id first_ipc_mark_section  \
0   0                      A   
1   1                      G   
2   2                      A   
3   3                      A   
4   4                      D   

                                         description  \
0  ABSTRACT\n\n  The disclosure relates to a meth...   
1  ABSTRACT\n\n       A system and method are pro...   
2  ABSTRACT\n\nA media module 10 for use with at ...   
3  ABSTRACT\n\n          A support garment having...   
4  THERMALLY REACTIVE THERMOPLASTIC INTERMEDIATE ...   

                                  description_tokens  
0  [abstract, disclosur, relat, method, gradual, ...  
1  [abstract, system, method, provid, execut, mul...  
2  [abstract, media, modul, use, least, one, item...  
3  [abstract, support, garment, flexibl, panel, f...  
4  [thermal, reactiv, thermoplast, intermedi, pro...  
